# 로우
- Row는 스파크 객체이고 순서가 있는 필드 집합 객체이다.
- 순서가 있으므로 Row의 각 필드를 0부터 시작하는 인덱스로 접근할 수 있다.

In [1]:
from pyspark.sql import Row
blog_row = Row(6, 'Reynold', 'Xin', 'http', 255568, '3/2/2015', ['twit','LinkedIn'])

In [2]:
# 인덱스로 개별 아이템에 접근한다
blog_row[1]

'Reynold'

Row 객체들은 빠른 탐색을 위해 데이터 프레임으로 만들어 사용하기도 한다.

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('practice').getOrCreate()

23/09/06 16:33:50 WARN Utils: Your hostname, minseok-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/09/06 16:33:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/06 16:33:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/06 16:33:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
rows = [Row('MZ', 'CA'), Row('RX','CA')]
authors_df = spark.createDataFrame(rows, ['Authors', 'State'])
# authors_df.show()

# 데이터프레임을 만들어 show하면 에러발생
# 실제로는 파일에서 데이터 프레임을 읽어 들여야하는 상황이 일반적이다.

# 자주 쓰이는 데이터 프레임 작업들

## DataFrameReader / DataFrameWriter

In [5]:
# 읽을 파일은 28개의 컬럼과 4,380,660개 이상의 레코드가 있기 때문에
# 스키마를 정의하는 것이 효과적이다.
from pyspark.sql.types import *
fire_schema = StructType([StructField('CalNumber', IntegerType(),True),
                          StructField('UnitID', StringType(),True),
                          StructField('IncidentNumber', IntegerType(),True),
                          StructField('CallType', StringType(),True),
                          StructField('CallDate', StringType(),True),
                          StructField('WatchDate', StringType(),True),
                          StructField('CallFinalDisposition', StringType(),True),
                          StructField('AvailableDtTm', StringType(),True),
                          StructField('Address', StringType(),True),
                          StructField('City', StringType(),True),
                          StructField('Zipcode', IntegerType(),True),
                          StructField('Battalion', StringType(),True),
                          StructField('StationArea', StringType(),True),
                          StructField('Box', StringType(),True),
                          StructField('OriginalPriority', StringType(),True),
                          StructField('Priority', StringType(),True),
                          StructField('FinalPriority', IntegerType(),True),
                          StructField('ALSUnit', BooleanType(),True),
                          StructField('CallTypeGroup', StringType(),True),
                          StructField('NumAlarms', IntegerType(),True),
                          StructField('UnitType', StringType(),True),
                          StructField('UnitSequenceCallDispatch', IntegerType(),True),
                          StructField('FirePreventionDistrict', StringType(),True),
                          StructField('SupervisorDistrict', StringType(),True),
                          StructField('Neighborhood', StringType(),True),
                          StructField('Location', StringType(),True),
                          StructField('RowID', StringType(),True),
                          StructField('Delay', FloatType(),True)])

# DataFrameReader 인터페이스로 CSV파일을 읽는다.
sf_fire_file = 'sf-fire-calls.csv'
fire_df = spark.read.csv(sf_fire_file, header=True, schema=fire_schema)

DataFrameWriter의 기본포맷은 파케이이며 데이터 압축에 스내피(snappy) 압축을 쓴다.<br>
만약 데이터 프레임이 파케이로 쓰여졌다면 스키마는 파케이 메타데이터의 일부로 보존될 수 있다. 이런 경우 데이터 프레임으로 읽어 들일 때 수동으로 스키마를 적용할 필요가 없다. 

In [6]:
# 파케이로 저장
parquet_path = './파케이저장연습'
fire_df.write.format('parquet').save(parquet_path)

# 폴더로 저장된다.
# parquet_path는 폴더경로를 의미함
# 폴더안에 .parquet파일이 있음

23/09/06 16:34:55 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
23/09/06 16:34:59 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: CallNumber, UnitID, IncidentNumber, CallType, CallDate, WatchDate, CallFinalDisposition, AvailableDtTm, Address, City, Zipcode, Battalion, StationArea, Box, OriginalPriority, Priority, FinalPriority, ALSUnit, CallTypeGroup, NumAlarms, UnitType, UnitSequenceInCallDispatch, FirePreventionDistrict, SupervisorDistrict, Neighborhood, Location, RowID, Delay
 Schema: CalNumber, UnitID, IncidentNumber, CallType, CallDate, WatchDate, CallFinalDisposition, AvailableDtTm, Address, City, Zipcode, Battalion, StationArea, Box, OriginalPriority, Priority, FinalPriority, ALSUnit, CallTypeGroup, NumAlarms, UnitType, UnitSequenceCallDispatch, FirePreventionDistrict, SupervisorDistrict, Neighborhood, Location, RowID, Delay
Expected: Cal

하이브 메타스토어에 메타데이터로 등록되는 테이블로 저장할 수 있다.

In [8]:
parquet_table = 'parquet_table'
fire_df.write.format('parquet').saveAsTable(parquet_table)

# 마찬가지로 parquet_table는 폴더이름을 의미함

23/09/06 16:39:03 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: CallNumber, UnitID, IncidentNumber, CallType, CallDate, WatchDate, CallFinalDisposition, AvailableDtTm, Address, City, Zipcode, Battalion, StationArea, Box, OriginalPriority, Priority, FinalPriority, ALSUnit, CallTypeGroup, NumAlarms, UnitType, UnitSequenceInCallDispatch, FirePreventionDistrict, SupervisorDistrict, Neighborhood, Location, RowID, Delay
 Schema: CalNumber, UnitID, IncidentNumber, CallType, CallDate, WatchDate, CallFinalDisposition, AvailableDtTm, Address, City, Zipcode, Battalion, StationArea, Box, OriginalPriority, Priority, FinalPriority, ALSUnit, CallTypeGroup, NumAlarms, UnitType, UnitSequenceCallDispatch, FirePreventionDistrict, SupervisorDistrict, Neighborhood, Location, RowID, Delay
Expected: CalNumber but found: CallNumber
CSV file: file:///home/minseok/spark-3.4.1-bin-hadoop3/python/Learning_Spark/Chapter3/sf-fire-calls.csv


## 프로젝션 / 필터

In [10]:
from pyspark.sql.functions import col
few_fire_df = (fire_df
               .select('IncidentNumber', 'AvailableDtTm', 'CallType')
               .where(col('CallType') != 'Medical Incident'))
few_fire_df.show(5, truncate=False)

+--------------+----------------------+--------------+
|IncidentNumber|AvailableDtTm         |CallType      |
+--------------+----------------------+--------------+
|2003235       |01/11/2002 01:51:44 AM|Structure Fire|
|2003250       |01/11/2002 04:16:46 AM|Vehicle Fire  |
|2003259       |01/11/2002 06:01:58 AM|Alarms        |
|2003279       |01/11/2002 08:03:26 AM|Structure Fire|
|2003301       |01/11/2002 09:46:44 AM|Alarms        |
+--------------+----------------------+--------------+
only showing top 5 rows



화재 신고로 기록된 CallType 종류가 몇 가지인지 알고 싶다면?

In [13]:
# countDistinct()를 써서 신고 타입의 개수를 되돌려 준다.
# pandas의 nunique()
from pyspark.sql.functions import *
(fire_df
 .select('CallType')
 .where(col('CallType').isNotNull())
 .agg(countDistinct('CallType').alias('DistinctCallTypes'))
 .show())

+-----------------+
|DistinctCallTypes|
+-----------------+
|               30|
+-----------------+



Null이 아닌 신고 타입의 목록?

In [14]:
(fire_df
 .select('CallType')
 .where(col('CallType').isNotNull())
 .distinct()
 .show(10, False))

+-----------------------------------+
|CallType                           |
+-----------------------------------+
|Elevator / Escalator Rescue        |
|Marine Fire                        |
|Aircraft Emergency                 |
|Confined Space / Structure Collapse|
|Administrative                     |
|Alarms                             |
|Odor (Strange / Unknown)           |
|Citizen Assist / Service Call      |
|HazMat                             |
|Watercraft in Distress             |
+-----------------------------------+
only showing top 10 rows



## 칼럼의 이름 변경 및 추가 삭제

일단은 StructField를 써서 스키마 내에서 원하는 칼럼 이름들을 지정하면 결과 데이터 프레임에서 원하는 대로 칼럼 이름이 출력된다. 하지만 이것은 데이터 소스의 칼럼 이름을 무시하고 원하는 이름으로 읽어 오는 경우이므로 '변경'과는 조금 의미가 다를 수 있다.<br>
다른 방법으로는 withColumnRenamed() 함수로 변경할 수 있다. <br>
'Delay'칼럼을 'ResponseDelayedinMins'로 바꾼 후 5분 이상 걸린 응답시간만 출력한다. <br>
이 경우에도 데이터 프레임 원본을 유히한 채로 컬럼 이름이 변경된 새로운 데이터 프레임을 받아 온다.

In [15]:
new_fire_df = fire_df.withColumnRenamed('Delay','ResponseDelayedinMins')
(new_fire_df
 .select('ResponseDelayedinMins')
 .where(col('ResponseDelayedinMins') > 5)
 .show(5, False))

+---------------------+
|ResponseDelayedinMins|
+---------------------+
|5.35                 |
|6.25                 |
|5.2                  |
|5.6                  |
|7.25                 |
+---------------------+
only showing top 5 rows



날짜 타입으로 변경
- to_timestamp
- to_date

In [17]:
fire_ts_df = (new_fire_df
              .withColumn('IncidentDate', to_timestamp(col('CallDate'), 'MM/dd/yyyy'))
              .drop('CallDate')
              # 'CallDate' 칼럼값을 형식이 'MM/dd/yyyy'인 timestamp유형으로 바꿔 'IncidentDate' 컬럼으로 생성
              # 기존 컬럼은 제거
              .withColumn('OnWatchDate', to_timestamp(col('WatchDate'), 'MM/dd/yyyy'))
              .drop('WatchDate')
              .withColumn('AvailableDtTs', to_timestamp(col('AvailableDtTm'), 'MM/dd/yyyy hh:mm:ss a'))
              .drop('AvailableDtTm'))

# 변환된 칼럼들을 가져온다.
(fire_ts_df
 .select('IncidentDate', 'OnWatchDate', 'AvailableDtTs')
 .show(5, False))

+-------------------+-------------------+-------------------+
|IncidentDate       |OnWatchDate        |AvailableDtTs      |
+-------------------+-------------------+-------------------+
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 01:51:44|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 03:01:18|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 02:39:50|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 04:16:46|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 06:01:58|
+-------------------+-------------------+-------------------+
only showing top 5 rows



날짜/시간 칼럼을 가지게 되었으므로 이후에 탐색을 할 때는 pyspark.sql.functions에서 dayofmonth(), dayofyear(), dayofweek() 같은 함수들을 써서 질의할 수 있다.<br>
몇 년 동안의 소방서 호출이 포함되어 있는지 확인한다.

In [18]:
(fire_ts_df
 .select(year('IncidentDate'))
 # IncidentDate 칼럼의 연도를 가져온다.
 .distinct()
 .orderBy(year('IncidentDate'))
 # select(year('IncidentDate'))로 인해 칼럼이름이 year('IncidentDate')이다.
 # 따라서 orderBy 메소드 인자로 칼럼이름인 year('IncidentDate')를 넣었다.
 .show())

+------------------+
|year(IncidentDate)|
+------------------+
|              2000|
|              2001|
|              2002|
|              2003|
|              2004|
|              2005|
|              2006|
|              2007|
|              2008|
|              2009|
|              2010|
|              2011|
|              2012|
|              2013|
|              2014|
|              2015|
|              2016|
|              2017|
|              2018|
+------------------+



## 집계연산

가장 흔한 형태의 신고는?

In [20]:
(fire_ts_df
 .select('CallType')
 .where(col('CallType').isNotNull())
 .groupBy('CallType')
 .count()
 .orderBy('count', ascending=False)
 .show(n=10, truncate=False))

+-------------------------------+------+
|CallType                       |count |
+-------------------------------+------+
|Medical Incident               |113794|
|Structure Fire                 |23319 |
|Alarms                         |19406 |
|Traffic Collision              |7013  |
|Citizen Assist / Service Call  |2524  |
|Other                          |2166  |
|Outside Fire                   |2094  |
|Vehicle Fire                   |854   |
|Gas Leak (Natural and LP Gases)|764   |
|Water Rescue                   |755   |
+-------------------------------+------+
only showing top 10 rows



In [22]:
'''
데이터 프레임 API는 collect() 함수를 제공하지만 극단적으로 큰 데이터 프레임에서는 메모리 부족 예외(OOM)를 발생시킬 수 있기
때문에 자원도 많이 쓰고 위험하다. collect()는 전체 데이터 프레임 혹은 데이터세트의 모든 Row 객체 모음을 되돌려 준다.
(리스트 형태로 되돌려줌)
몇 개의 Row 결과만 보고 싶다면 최초 n개의 Row 객체만 되돌려 주는 take(n) 함수를 쓰는 것이 훨씬 좋다.
'''

fire_ts_df.take(2)

23/09/06 17:34:49 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: CallNumber, UnitID, IncidentNumber, CallType, CallDate, WatchDate, CallFinalDisposition, AvailableDtTm, Address, City, Zipcode, Battalion, StationArea, Box, OriginalPriority, Priority, FinalPriority, ALSUnit, CallTypeGroup, NumAlarms, UnitType, UnitSequenceInCallDispatch, FirePreventionDistrict, SupervisorDistrict, Neighborhood, Location, RowID, Delay
 Schema: CalNumber, UnitID, IncidentNumber, CallType, CallDate, WatchDate, CallFinalDisposition, AvailableDtTm, Address, City, Zipcode, Battalion, StationArea, Box, OriginalPriority, Priority, FinalPriority, ALSUnit, CallTypeGroup, NumAlarms, UnitType, UnitSequenceCallDispatch, FirePreventionDistrict, SupervisorDistrict, Neighborhood, Location, RowID, Delay
Expected: CalNumber but found: CallNumber
CSV file: file:///home/minseok/spark-3.4.1-bin-hadoop3/python/Learning_Spark/Chapter3/sf-fire-calls.csv


[Row(CalNumber=20110016, UnitID='T13', IncidentNumber=2003235, CallType='Structure Fire', CallFinalDisposition='Other', Address='2000 Block of CALIFORNIA ST', City='SF', Zipcode=94109, Battalion='B04', StationArea='38', Box='3362', OriginalPriority='3', Priority='3', FinalPriority=3, ALSUnit=False, CallTypeGroup=None, NumAlarms=1, UnitType='TRUCK', UnitSequenceCallDispatch=2, FirePreventionDistrict='4', SupervisorDistrict='5', Neighborhood='Pacific Heights', Location='(37.7895840679362, -122.428071912459)', RowID='020110016-T13', ResponseDelayedinMins=2.950000047683716, IncidentDate=datetime.datetime(2002, 1, 11, 0, 0), OnWatchDate=datetime.datetime(2002, 1, 10, 0, 0), AvailableDtTs=datetime.datetime(2002, 1, 11, 1, 51, 44)),
 Row(CalNumber=20110022, UnitID='M17', IncidentNumber=2003241, CallType='Medical Incident', CallFinalDisposition='Other', Address='0 Block of SILVERVIEW DR', City='SF', Zipcode=94124, Battalion='B10', StationArea='42', Box='6495', OriginalPriority='3', Priority='3

## 그 외 일반적인 데이터 프레임 연산들

In [23]:
import pyspark.sql.functions as F
(fire_ts_df
 .select(F.sum('NumAlarms'), F.avg('ResponseDelayedinMins'),
         F.min('ResponseDelayedinMins'), F.max('ResponseDelayedinMins'))
 .show())

+--------------+--------------------------+--------------------------+--------------------------+
|sum(NumAlarms)|avg(ResponseDelayedinMins)|min(ResponseDelayedinMins)|max(ResponseDelayedinMins)|
+--------------+--------------------------+--------------------------+--------------------------+
|        176170|         3.892364154521585|               0.016666668|                   1844.55|
+--------------+--------------------------+--------------------------+--------------------------+



stat(), describe(), correlation(), covariance(), sampleBy(), approxQuantile(), frequentItems() 등의 API 문서를 읽어보기 바람.